## Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайтов Superjob и HH. Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). 

Получившийся список должен содержать в себе минимум:
* Наименование вакансии.
* Предлагаемую зарплату (отдельно минимальную, максимальную и валюту).
* Ссылку на саму вакансию.
* Сайт, откуда собрана вакансия.
По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [57]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import re
import pandas as pd

In [58]:
#https://www.superjob.ru/vacancy/search/?keywords=Python&geo%5Bt%5D%5B0%5D=4
#https://hh.ru/search/vacancy?area=1&fromSearchLine=true&st=searchVacancy&text=Python&from=suggest_post
#https://hh.ru/search/vacancy?area=1&fromSearchLine=true&st=searchVacancy&text=all&from=suggest_post

In [69]:
main_link = 'https://hh.ru/search/vacancy'
hhparams = {'area': '1', 'fromSearchLine': 'true', 'st': 'searchVacancy', 'text': 'Курьер', 'from': 'suggest_post'}
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.80 Safari/537.36'}
response = requests.get(main_link, params=hhparams, headers=headers)
vacancies = []

if response.status_code == 200:
    dom = bs(response.text, 'html.parser')
    vacancy_list = dom.find_all('div', {'class': 'vacancy-serp-item__row vacancy-serp-item__row_header'})
    
for vacancy in vacancy_list:
    vacancy_data = {}
    vacancy_name = vacancy.find('a')
    vacancy_link = main_link + vacancy_name['href']
    vacancy_name = vacancy.find('a').getText()
        #salary = vacancy.find_all('div', {'Class': 'vacancy-serp-item__sidebar'})
    vacancy_salary = vacancy.find('span', {'data-qa':'vacancy-serp__vacancy-compensation'})
        #vacancy_salary = salary.find('span').getText()
    if not vacancy_salary:
            salary_min = None
            salary_max = None
            salary_currency = None
    else:
        vacancy_salary = vacancy_salary.getText().replace(u'\xa0', u'')
        vacancy_salary = re.split(r'\s|-', vacancy_salary)
        if vacancy_salary[0] == 'до':
            salary_min = None
            salary_max = float(vacancy_salary[1])
            salary_currency = vacancy_salary[2]
        elif vacancy_salary[0] == 'от':
            salary_max = None
            salary_min = float(vacancy_salary[1])
            salary_currency = vacancy_salary[2]
        else:
            salary_max = float(vacancy_salary[1])
            salary_min = float(vacancy_salary[0])
            salary_currency = vacancy_salary[2]
        
    vacancy_data['name'] = vacancy_name
    vacancy_data['link'] = vacancy_link
        #vacancy_data['salary'] = vacancy_salary
    vacancy_data['salary_min'] = salary_min
    vacancy_data['salary_max'] = salary_max
    vacancy_data['currency'] = salary_currency
    vacancies.append(vacancy_data)

pprint(vacancies)
       

[{'currency': 'руб.',
  'link': 'https://hh.ru/search/vacancyhttps://hh.ru/vacancy/38979991',
  'name': 'Пеший курьер',
  'salary_max': None,
  'salary_min': 40000.0},
 {'currency': 'руб.',
  'link': 'https://hh.ru/search/vacancyhttps://hh.ru/vacancy/39522415',
  'name': 'Курьер в юридический отдел',
  'salary_max': None,
  'salary_min': 30000.0},
 {'currency': 'руб.',
  'link': 'https://hh.ru/search/vacancyhttps://hh.ru/vacancy/40015093',
  'name': 'Менеджер/Курьер (по доставке банковских карт)',
  'salary_max': None,
  'salary_min': 55000.0},
 {'currency': 'руб.',
  'link': 'https://hh.ru/search/vacancyhttps://hh.ru/vacancy/38741571',
  'name': 'Пеший курьер',
  'salary_max': 50000.0,
  'salary_min': 40000.0},
 {'currency': 'руб.',
  'link': 'https://hh.ru/search/vacancyhttps://hh.ru/vacancy/39203668',
  'name': 'Пеший курьер - свободный график',
  'salary_max': 40000.0,
  'salary_min': None},
 {'currency': 'руб.',
  'link': 'https://hh.ru/search/vacancyhttps://hh.ru/vacancy/37455026

In [70]:
df = pd.DataFrame(vacancies)
df[0:20]

,name,link,salary_min,salary_max,currency
0,Пеший курьер,https://hh.ru/search/vacancyhttps://hh.ru/vaca...,40000.0,NaN,руб.
1,Курьер в юридический отдел,https://hh.ru/search/vacancyhttps://hh.ru/vaca...,30000.0,NaN,руб.
2,Менеджер/Курьер (по доставке банковских карт),https://hh.ru/search/vacancyhttps://hh.ru/vaca...,55000.0,NaN,руб.
3,Пеший курьер,https://hh.ru/search/vacancyhttps://hh.ru/vaca...,40000.0,50000.0,руб.
4,Пеший курьер - свободный график,https://hh.ru/search/vacancyhttps://hh.ru/vaca...,NaN,40000.0,руб.
5,Водитель-курьер,https://hh.ru/search/vacancyhttps://hh.ru/vaca...,90000.0,100000.0,руб.
6,Курьер,https://hh.ru/search/vacancyhttps://hh.ru/vaca...,50000.0,90000.0,руб.
7,Водитель-курьер на личном автомобиле,https://hh.ru/search/vacancyhttps://hh.ru/vaca...,75000.0,NaN,руб.
8,Пеший курьер,https://hh.ru/search/vacancyhttps://hh.ru/vaca...,30000.0,37500.0,руб.
9,Водитель-курьер ⁢по доставке заказов (вахта 14...,https://hh.ru/search/vacancyhttps://hh.ru/vaca...,70000.0,NaN,руб.


In [107]:
def _superjob(vacancy):
    vacancy_date = []
    params = {'keywords': vacancy, 'profession_only': '1', 'geo[c][0]': '15', 'geo[c][1]': '1', 'geo[c][2]': '9', 'page': ''}
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.80 Safari/537.36'}
    link = 'https://www.superjob.ru/vacancy/search/' 
    html = requests.get(link, params=params, headers=headers)
    if html.ok:
        parsed_html = bs(html.text,'html.parser')
        page_block = parsed_html.find('a', {'class': 'f-test-button-1'})
    if not page_block:
        last_page = 1
    else:
        page_block = page_block.findParent()
        last_page = int(page_block.find_all('a')[-2].getText())
    for page in range(0, last_page + 1):
        params['page'] = page
        html = requests.get(link, params=params, headers=headers)
        if html.ok:
            parsed_html = bs(html.text,'html.parser')
            vacancy_items = parsed_html.find_all('div', {'class': 'f-test-vacancy-item'})      
            for item in vacancy_items:
                vacancy_date.append(_item_superjob(item))            
    return vacancy_date

In [108]:
def _item_superjob(item):
    vacancy_date = {}
# vacancy_name
    vacancy_name = item.find_all('a')
    if len(vacancy_name) > 1:
        vacancy_name = vacancy_name[-2].getText()
    else:
        vacancy_name = vacancy_name[0].getText()
    vacancy_date['vacancy_name'] = vacancy_name
    
# company_name
    company_name = item.find('span', {'class': 'f-test-text-vacancy-item-company-name'})
    if not company_name:
        company_name = item.findParent().find('span', {'class': 'f-test-text-vacancy-item-company-name'})
    else:
        company_name = company_name.getText()
    vacancy_date['company_name'] = company_name
    
#salary
    salary = item.find('span', {'class': 'f-test-text-company-item-salary'}) \
                  .findChildren()
    if not salary:
        salary_min = None
        salary_max = None
        salary_currency = "руб."
    else:
        salary_currency = salary[-1].getText()
        is_check_sarary = item.find('span', {'class': 'f-test-text-company-item-salary'}).getText() \
                                .replace(u'\xa0', u' ') \
                                .split(' ', 1)[0]
        if is_check_sarary == 'до ':
            salary_min = None
            salary_max = int(salary[0].getText() \
                                        .replace(u'\xa0', u''))
        elif is_check_sarary == 'от ':
            salary_min = int(salary[0].getText() \
                                         .replace(u'\xa0', u''))
            salary_max = None
        elif is_check_sarary == 'По договорённости':
            salary_min = None
            salary_max = None
        else:
            salary_min = None
            salary_max = None                 
    vacancy_date['salary_min'] = salary_min
    vacancy_date['salary_max'] = salary_max
    vacancy_date['salary_currency'] = salary_currency
    
# link
    vacancy_link = item.find_all('a')
    if len(vacancy_link) > 1:
        vacancy_link = vacancy_link[-2]['href']
    else:
        vacancy_link = vacancy_link[0]['href']   
    vacancy_date['vacancy_link'] = f'https://www.superjob.ru{vacancy_link}'
# site
    vacancy_date['site'] = 'www.superjob.ru'
    return vacancy_date

In [109]:
def _vacancy(vacancy):
    vacancy_date = []
    vacancy_date.extend(_superjob(vacancy))
    df = pd.DataFrame(vacancy_date)
    return df

In [110]:
vacancy = 'Курьер'

In [111]:
df = _vacancy(vacancy)

In [112]:
df[1:20]

,vacancy_name,company_name,salary_min,salary_max,salary_currency,vacancy_link,site
1,,YAMI YAMI,None,None,месяц,https://www.superjob.ru/clients/yami-yami-2878...,www.superjob.ru
2,,Яндекс.Еда (Центр.найма курьеров),None,None,день,https://www.superjob.ru/clients/centr-najma-39...,www.superjob.ru
3,,SVL-Group,None,None,месяц,https://www.superjob.ru/clients/svl-group-2719...,www.superjob.ru
4,,Fastzila,None,None,месяц,https://www.superjob.ru/clients/fastzila-29381...,www.superjob.ru
5,,Амбер,None,None,месяц,https://www.superjob.ru/clients/amber-3035414....,www.superjob.ru
6,"Водитель-экспедитор/курьер (на своем авто, с р...",КАРДИОСТАР,None,None,месяц,https://www.superjob.ru/vakansii/voditel-ekspe...,www.superjob.ru
7,,Яндекс. Такси (Центр Найма Водителей),None,None,месяц,https://www.superjob.ru/clients/yandeks-taksi-...,www.superjob.ru
8,,Купишуз (Lamoda Group),None,None,месяц,https://www.superjob.ru/clients/kupishuz-lamod...,www.superjob.ru
9,Курьер-техник,ООО НАШ ПОРТ,None,None,месяц,https://www.superjob.ru/vakansii/kurer-tehnik-...,www.superjob.ru
10,iGooods,iGooods,None,None,месяц,https://www.superjob.ru/clients/ajguds-ru-2667...,www.superjob.ru


## END